## NHL API

Данные 2015-2019 взяты с Каггла https://www.kaggle.com/kapastor/nhl-data    
Остальные данные 2010-2015 и 2020 скачаны с NHL   
Ниже представлена предоработка файлов pkl и сохранение их в csv для последующей работы

In [1]:
import numpy as np 
import pandas as pd
import pickle    
import warnings

In [3]:
def players_extract(row):
    """
    Разбирает столбец на действующих в событии хоккеров
    """
    # main player
    player_name = row['players'][0]['player']['fullName']
    player_id = row['players'][0]['player']['id']
    
    if row['event'] == 'Faceoff':
        row['player_type'] = 'faceoff_winner'
        
    if row['event'] == 'Takeaway':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'takeaway'
        
    elif row['event'] == 'Blocked Shot':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'blocker'
        
    elif row['event'] == 'Shot':
        for player in row['players']:
            if player['playerType'] == 'Shooter':
                row['player'] = player['player']['fullName']
                #row['id'] = player['player']['id']
                row['player_type'] = 'shooter'
            elif player['playerType'] == 'Goalie':
                row['goalie'] = player['player']['fullName']
        
    elif row['event'] == 'Missed Shot':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'shooter'
        
    elif row['event'] == 'Hit':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'hitter'
        
    elif row['event'] == 'Giveaway':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'puck_loser'
    elif row['event'] == 'Goal':
        assists = []
        assists_id = []
        #assist_count=1
        for player in row['players']:
            if player['playerType'] == 'Scorer':
                #row['player'] = player['player']['fullName']
                #row['id'] = player['player']['id']
                row['player_type'] = 'scorer'
            elif player['playerType'] == 'Assist':
                assists.append(player['player']['fullName'])
                assists_id.append(player['player']['id'])
            elif player['playerType'] == 'Goalie':
                row['goalie'] = player['player']['fullName']
        row['assists'] = assists
        row['assists_ids'] = assists_id
    elif row['event'] == 'Penalty':
        #row['player'] = player_name
        #row['id'] = player_id
        row['player_type'] = 'penalty_on'
    return row



def events_prep(df):
    """
    Производит обработку датафрейма df(events) - с игровыми событиями
    """
    
    # создаем столбец события
    df['event'] = df['result'].apply(lambda x: x['event'])

    # добавляем описание
    df['description'] = df['result'].apply(lambda x: x['description'])

    # убираем лишние события
    df = df.loc[df.event.isin(events)]
    df.reset_index(drop=True, inplace=True) # обновляем нумерацию индекса

    # фиксируем тип броска
    df.loc[(df.event.isin(['Goal', 'Shot']))&(df[
        'result'].astype('str').str.contains('secondaryType')), 'shot_type'] = df.loc[
        (df.event.isin(['Goal', 'Shot']))&(df['result'].astype('str').str.contains(
            'secondaryType')), 'result'].apply(lambda x: x['secondaryType']) 

    # заполняем пробелы (в т.ч. в пустые ворота) - самым популярным кистевым броском - Wrist Shot
    df.loc[(df.event.isin(['Goal', 'Shot']))&(~df[
        'result'].astype('str').str.contains('secondaryType')), 'shot_type'] = 'Wrist Shot'

    df.loc[df.event=='Goal','team_str'] = df.loc[df.event=='Goal', 'result'].apply(lambda x: x['strength']['name'])
    
    # Обработка about

    # Номер периода
    df.loc[:, 'period'] = df.loc[:, 'about'].apply(lambda x: x['period'])

    # Время события
    df.loc[:, 'periodTime'] = df.loc[:, 'about'].apply(lambda x: x['periodTime'])

    # текущий счет
    df.loc[:, 'goals'] = df.loc[:, 'about'].apply(lambda x: x['goals'])

    # удаляем result, about
    df = df.drop(['result', 'about'], axis=1)
    
    df.loc[:, 'team'] = df.loc[:, 'team'].apply(lambda x: x['name'])

    df.loc[:, 'player'] = df.loc[:, 'players'].apply(lambda x: x[0]['player']['fullName'])
    df.loc[:, 'player_id'] = df.loc[:, 'players'].apply(lambda x: x[0]['player']['id'])

    df = df.apply(players_extract, axis=1)

    # players больше не нужен
    df.drop('players', axis=1, inplace=True)
    
    return df


def players_prep(pl):
    """ обработка датафрейма с данными игроков, оставляет последние значения"""
    
    pl.loc[~pl['currentTeam'].isna(), 'currentTeam'] = pl.loc[
        ~pl['currentTeam'].isna(), 'currentTeam'].apply(lambda x: x['name'])

    pl['currentTeam'].fillna('Free Agent', inplace=True)

    # основная позиция
    pl['primaryPosition'] = pl['primaryPosition'].apply(lambda x: x['abbreviation'])

    pl.drop(['link', 'firstName', 'lastName', 'birthCity', 'birthStateProvince', 'birthCountry', 'active', 
         'alternateCaptain', 'captain', 'rookie', 'rosterStatus'], axis=1, inplace=True)

    pl.drop_duplicates(subset=['fullName'], keep='last', inplace=True)
    
    return pl


def stats_prep(st):
    """Обработка статистики игроков, на выходе выдает 2 ДФ - игроки и вратари"""
    st = st.loc[st.position!='Unknown']

    # Разбиваем на 2 ДФ
    goalie_stats = st.loc[st.position=='Goalie']
    players_stats = st.loc[st.position!='Goalie']

    # дергаем словари
    players_stats['stats'] = players_stats['stats'].apply(lambda x: x['skaterStats'])
    goalie_stats['stats'] = goalie_stats['stats'].apply(lambda x: x['goalieStats'])

    # Создаем списки столбцов
    stats_pl_cols = players_stats.stats.apply(pd.Series).fillna(0).columns
    stats_gl_cols = goalie_stats.stats.apply(pd.Series).fillna(0).columns

    # Добавляем столбцы из словарей
    players_stats[stats_pl_cols] = players_stats.stats.apply(pd.Series)
    goalie_stats[stats_gl_cols] = goalie_stats.stats.apply(pd.Series)

    # дропаем столбец 
    players_stats.drop(['stats'], axis=1, inplace=True)
    goalie_stats.drop(['stats'], axis=1, inplace=True)
    
    return players_stats, goalie_stats

In [14]:
def pickle_data(game_data):
    """
    Получает на вход game_data (pickle файл)
    возвращает датафреймы с событиями, игроками, статистикой и т.д.
    """
    
    df = pd.DataFrame() # события
    st = pd.DataFrame() # статистика
    pl = pd.DataFrame() # игроки
    coaches = pd.DataFrame() # тренеры
    games = pd.DataFrame() # даты игр

    # смотрим какие типы событий есть в API 
    for data in game_data:
        if 'liveData' not in data: # Make sure the data is valid
            continue
        game_id = data['gamePk']
        # события
        game_plays = pd.DataFrame(data['liveData']['plays']['allPlays'])
        game_plays['game_id'] = None
        game_plays.loc[:, 'game_id'] = game_id
        df = pd.concat([df, game_plays])
    
        # roster
        game_roster = pd.DataFrame(data['gameData']['players']).T
        pl = pd.concat([pl, game_roster])  
        
        # информация об играх, даты и команды
        game_info = pd.DataFrame(data['gameData'])[['game', 'datetime', 'teams']].dropna(
            thresh=1).T.fillna(method='ffill').dropna()[['pk', 'dateTime', 'away', 'home']]

        game_info['away'] = game_info['away'].apply(lambda x: x['name'])
        game_info['home'] = game_info['home'].apply(lambda x: x['name'])
        
        games = pd.concat([games, game_info])
        
        # статистика игроков
        for team in data['liveData']['boxscore']['teams']: # home/away
            player_stats = pd.DataFrame(data['liveData']['boxscore']['teams'][team]['players']).T#['stats']['skaterStats'])
            if len(player_stats)>0:
                player_stats['game_id'] = None
                player_stats.loc[:, 'game_id'] = game_id
                player_stats['person'] = player_stats['person'].apply(lambda x: x['id'])
                player_stats['position'] = player_stats['position'].apply(lambda x: x['name'])
                st = pd.concat([st, player_stats])
        
            # тренер в данной игре
            if len(data['liveData']['boxscore']['teams'][team]['coaches'])>0:
                team_pl = data['liveData']['boxscore']['teams'][team]['team']['name']
                coach = data['liveData']['boxscore']['teams'][team]['coaches'][0]['person']['fullName']
                coach_d =  {'game_id': game_id, 'team': team_pl, 'coach': coach}
                app_df = pd.DataFrame(coach_d, columns = ['game_id', 'team', 'coach'], index=[0])
                coaches = pd.concat([coaches, app_df]) 
    
    # preprocessing
    df = events_prep(df)
    print('df prep..ok')
    pl = players_prep(pl)
    print('players prep..ok')
    st_pl, st_gl = stats_prep(st)
    print('stats prep...ok')
    
    return df, pl, st_pl, st_gl, coaches, games
    

Пример обработки на сезоне-2018

In [6]:
season=2018
file_path = './input/nhl-data/' + str(season) + 'FullDataset.pkl'
print(file_path)
with open(file_path, 'rb') as f:
    game_data = pickle.load(f)

./input/nhl-data/2018FullDataset.pkl


### NHL API

In [8]:
# смотрим ключи
game_data[3].keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

#### Основные ключи  в API  
**copyright** - то, что защищено NHL это и так ясно. Не нужно  
**gamePk** - id игры, год, тип-сезон и порядковый номер  
**link** - ссылка в API   
**metaData** - временной штамп  
**gameData** - составы команд и официальная информация о командах, арене и пр.  
**liveData** - Ход матча и игровые события  

Из этих интересует только id, составы команд и игровая информация.  
Составы команд нужны, чтобы понять, на какой позиции играл игрок в данном матче. 

#### gameData - составы команд  
**game** - информация о игре *pk* - id игры, *season* - сезон, *type* - R(Regular)  
**datetime** - старт и финиш игры   
**status**, **venue** - некая служебная информация  
**players** - составы команд  
В составах команд есть ссылки на API / players. Но проще создать датафрейм из составов команд, а не парсить всех игроков

Из этого нужны id игры и составы команд

In [9]:
game_data[3]['gameData']['game']

{'pk': 2018020003, 'season': '20182019', 'type': 'R'}

In [12]:
pd.DataFrame(game_data[3]['gameData']['players']).T.head(2)

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthCountry,...,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,currentTeam,primaryPosition,birthStateProvince
ID8470626,8470626,Loui Eriksson,/api/v1/people/8470626,Loui,Eriksson,21,1985-07-17,34,Gothenburg,SWE,...,179,True,False,False,False,L,Y,"{'id': 23, 'name': 'Vancouver Canucks', 'link'...","{'code': 'L', 'name': 'Left Wing', 'type': 'Fo...",NaN
ID8470966,8470966,Mark Giordano,/api/v1/people/8470966,Mark,Giordano,5,1983-10-03,36,Toronto,CAN,...,200,True,False,True,False,L,Y,"{'id': 20, 'name': 'Calgary Flames', 'link': '...","{'code': 'D', 'name': 'Defenseman', 'type': 'D...",ON


#### liveData - ход матча и игровые события    
1 **plays**  
1.1 **allPlays**  - все события  
1.2 **scoringPlays**  -  список с номерами голов  
1.3 **penaltyPlays**  - список с номерами удалений  
1.4 **playsByPeriod** - разделение событий по периодам  
1.5 **currentPlay**  - служебная инфа для онлайна   
<BR>
Так как нужна информация не только с голами, но и другой статистикой, то все кроме **allPlays** - не нужно    

2 **linescore**  - статистика команд по броскам и голам за периоды и игру  
3 **boxscore**  - индивидуальная статистика игроков, информация о тренерах  
3.1 **teams** - информация о командах (home/away) и officials - тренерская бригада   
    3.1.1 **team** - команда и id команды  
    3.1.2 **teamStats** - общая статистика команды (броски, блоки, PP и другое)   
    3.1.3 **players** - индивидуальная статистика игроков, включая время на льду общее PP SH  
    3.1.4 **goalies** - id вратарей, которые играли  
    3.1.5 **players** - id игроков, которые играли  
    **onIce, onIcePlus, scratches, penaltyBox** - игроки в запасе, смены, выносливость ?  и другое  
    3.1.-1 **coaches** - тренеры   
4. **decisions** - три звезды матча, виннер/лузер среди вратарей

Какие данные в итоге нужны?  

----

**gamePk** - id игры и сезона

**gameData** - **game** - id игры сезона  
**gameData** - **players** База данных игроков (физ. параметры и пр.)  
**liveData** - **plays** - **allPlays**  - все игровые события  
**liveData** - **teams** - **team** - id команды   
**liveData** - **teams** - **players** - статистика игроков  
**liveData** - **teams** - **coaches** - тренеры 

Вся обработка производится функцией pickle_data, например  

*df, pl, st_pl, st_gl, coaches, games = pickle_data(game_data)*

## 1.1 NHL.API - gamedata-game и liveData-plays-allPlays 

In [5]:
#df = pd.read_csv('df2016.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237827 entries, 0 to 237826
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   assists      6817 non-null    object
 1   assists_ids  6817 non-null    object
 2   coordinates  237827 non-null  object
 3   description  237827 non-null  object
 4   event        237827 non-null  object
 5   game_id      237827 non-null  int64 
 6   goalie       73770 non-null   object
 7   goals        237827 non-null  object
 8   period       237827 non-null  int64 
 9   periodTime   237827 non-null  object
 10  player       237827 non-null  object
 11  player_id    237827 non-null  int64 
 12  player_type  237827 non-null  object
 13  players      237827 non-null  object
 14  shot_type    74061 non-null   object
 15  team         237827 non-null  object
 16  team_str     6817 non-null    object
dtypes: int64(3), object(14)
memory usage: 30.8+ MB


Разберем типы событий, которые есть в *NHL API*  (**event**)

------

##### События в атаке (команда владеет шайбой)
**Goal**  - гол!.     
**Shot**  - бросок в створ ворот (идет в статистику бросков)   
**Missed Shot** - бросок в сторону ворот, штангу, перекладину. Не идет в статистику.  
**Giveaway**  - потеря шайбы (пас сопернику) 

##### События в защите
**Blocked Shot** - заблокированный бросок  
**Hit**  -  силовой прием  
**Takeaway** - перехват шайбы  

**Penalty**  - относится к атаке, если правила нарушила атакующая команда или к защите, если оборонительная

##### Неигровые события
**Game End, Game Sheduled, Period End, Period Official, Period Ready, Period Start, Official Chellenge, Stoppage**   
Связаны с началом игры, периодов, остановками и пр. - они неинтересны

**Faceoff**  - достаточно официальной статистики, также не нужен

In [16]:
events = ['Blocked Shot',
         'Giveaway',
         'Goal',
         'Hit',
         'Missed Shot',
         'Penalty',
         'Shot',
         'Takeaway',
         'Faceoff']

## 1.2 NHL.API - gamedata-players

### Описание столбцов  
-----
**assists** и **assists_ids** - Список (list) и **id** ассистентов, если они есть  
**coordinates** - координаты события, если они нужны  
**description** - текстовое описание события  
**event** - типы события: гол, бросок, перехват и пр.  
**game_id** - **id** игры  
**goalie** - вратарь, если он есть в событии  
**goals** - счет (dict), после события  
**period** и **periodTime** - период и время периода  
**player**, **player_id** и **player_type**  - Имя игрока, id и роль игрока в событии  
**shot_type** - тип броска по воротам, если есть  
**team** - команда, к которой принадлежит игрок  
**team_str** - сила команда: равные составы или нет 

In [132]:
pl.sample(2)

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,currentTeam,primaryPosition
ID8467967,8467967,Chris Kelly,/api/v1/people/8467967,Chris,Kelly,22,1980-11-11,NaN,Toronto,ON,...,"6' 0""",190,False,NaN,NaN,False,L,N,Free Agent,C
ID8474683,8474683,Derek Grant,/api/v1/people/8474683,Derek,Grant,38,1990-04-20,30,Abbotsford,BC,...,"6' 3""",202,True,False,False,False,L,Y,Anaheim Ducks,C


In [141]:
pl.sample(2)

,id,fullName,primaryNumber,birthDate,currentAge,nationality,height,weight,shootsCatches,currentTeam,primaryPosition
ID8474056,8474056,P.K. Subban,76,1989-05-13,30,CAN,"6' 0""",210,R,Nashville Predators,D
ID8473588,8473588,Erik Condra,22,1986-08-06,33,USA,"5' 11""",188,R,Tampa Bay Lightning,RW


In [146]:
#df[['player', 'player_type']].merge(pl[['fullName', 'nationality']], left_on='player', right_on='fullName', how='right')

In [149]:
pl.describe(include='all').T

,count,unique,top,freq
id,1009,1004,8476406,2
fullName,1009,1009,Marian Gaborik,1
primaryNumber,994,94,15,25
birthDate,1009,889,1980-09-26,3
currentAge,908,22,27,110
nationality,1009,18,CAN,462
height,1009,16,"6' 1""",196
weight,1009,79,200,58
shootsCatches,1009,2,L,644
currentTeam,1009,32,Free Agent,101


### players - описание столбцов   
-----
**id** и **fullName** - id и ФИО игрока  
**primaryNumber** - игровой номер (просто так, нравятся игровые свитера с номерами)   
**birthDay**, **currentAge** - ДР и текущий возраст  
**height** и **weight** - рост и вес  
**shootCatches** - хват клюшки или ловушка (для вратаря)  
**currentTeam** - текущая команда  
**primaryPosition** - игровая позиция

## 1.3 NHL.API - livedata-players  - статистика игроков

#### Сводная статистика игроков

In [317]:
players_stats.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
person,44274,NaN,NaN,NaN,8.47437e+06,3274.71,8.44821e+06,8.47174e+06,8.47518e+06,8.47679e+06,8.48009e+06
jerseyNumber,44274,89,27,1397,NaN,NaN,NaN,NaN,NaN,NaN,NaN
position,44274,4,Defenseman,14802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_id,44274,NaN,NaN,NaN,2.01602e+09,355.066,2.01602e+09,2.01602e+09,2.01602e+09,2.01602e+09,2.01602e+09
timeOnIce,44274,1677,17:01,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN
assists,44274,NaN,NaN,NaN,0.253851,0.519751,0,0,0,0,4
goals,44274,NaN,NaN,NaN,0.151421,0.402174,0,0,0,0,4
shots,44274,NaN,NaN,NaN,1.67891,1.50522,0,1,1,2,12
hits,44274,NaN,NaN,NaN,1.20791,1.38884,0,0,1,2,14
powerPlayGoals,44274,NaN,NaN,NaN,0.0317342,0.181871,0,0,0,0,3


##### Описания столбцов
----
**position** - игровая позиция  
**game_id** - id игры  
**timeOnIce** - время на льду, мин  
**assists** - голевые пасы  
**goals** - голы  
**shots** - броски  
**hits** - силовые приемы  
**powerPlayGoals** - голы в большинстве  
**powerPlayAssists** - пасы в большинстве  
**penaltyMinutes** - удаления, мин  
**faceOffPct** - вбрасывания, %  
**faceOffWins** - выигранные вбрасывания  
**faceoffTaken** - всего вбрасываний  
**takeaways** - перехваты  
**giveaways** - потери шайбы  
**shortHandedGoals** - голы в меньшинстве  
**shortHandedAssists** - пасы в меньшинстве  
**blocked** - блокированные броски  
**plusMinus** - плюс-минус  
**evenTimeOnIce** время в равных составах  
**powerPlayTimeOnIce** - время в большинстве  
**shortHandedTimeOnIce** - время в меньшинстве 

#### Сводная статистика вратарей

In [318]:
goalie_stats.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
person,2657,NaN,NaN,NaN,8.47329e+06,2816.43,8.46614e+06,8.47122e+06,8.47358e+06,8.47568e+06,8.4795e+06
jerseyNumber,2657,21,31,484,NaN,NaN,NaN,NaN,NaN,NaN,NaN
position,2657,1,Goalie,2657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
game_id,2657,NaN,NaN,NaN,2.01602e+09,353.918,2.01602e+09,2.01602e+09,2.01602e+09,2.01602e+09,2.01602e+09
timeOnIce,2657,683,60:00,774,NaN,NaN,NaN,NaN,NaN,NaN,NaN
assists,2657,NaN,NaN,NaN,0.0131728,0.117291,0,0,0,0,2
goals,2657,NaN,NaN,NaN,0,0,0,0,0,0,0
pim,2657,NaN,NaN,NaN,0.0639819,0.588904,0,0,0,0,22
shots,2657,NaN,NaN,NaN,27.8683,8.72729,0,23,28,34,60
saves,2657,NaN,NaN,NaN,25.4565,8.55891,0,21,26,31,58


##### Описания столбцов
----
**position** - игровая позиция  
**game_id** - id игры  
**timeOnIce** - время на льду, мин  
**assists** - голевые пасы  
**goals** - голы  
**pim** - удаления в минутах  
**shots** - броски  
**saves** - спасения  
**powerPlaySaves** - спасения в большинстве  
**shortHandedSaves** - спасения в меньшинстве  
**evenSaves** - спасения в равных составах  
**shortHandedShotsAgainst** - броски в меньшинстве  
**evenShotsAgainst** - броски в равных составах  
**powerPlayShotsAgainst** - броски в большинстве  
**decision** - итог (победа/поражение)  
**savePercentage** - надежность в %  
**powerPlaySavePercentage** - надежность в большинстве  
**shortHandedSavePercentage** - надежность в меньшинстве  
**evenStrengthSavePercentage** - надежность в равных составах  

## 1.4 NHL.API - livedata-teams  -  команды

In [358]:
game_data[1]['liveData']['boxscore']['teams']['home']['team']

{'id': 9,
 'name': 'Ottawa Senators',
 'link': '/api/v1/teams/9',
 'abbreviation': 'OTT',
 'triCode': 'OTT'}

In [346]:
game_data[3]['liveData']['boxscore']['teams']['home']['coaches'][0]['person']['fullName']

'Todd McLellan'

In [383]:
pd.DataFrame({'game_id': [22333], 'team': ['Ottawa Senators']
              , 'coach': ['Todd McLellan']}, columns = ['game_id', 'team', 'coach'])

,game_id,team,coach
0,22333,Ottawa Senators,Todd McLellan


#### Прогоняем все файлы и сохраняем csv

In [ ]:
warnings.filterwarnings(action='once')

df_full = pd.DataFrame()
st_pl_full = pd.DataFrame()
st_gl_full = pd.DataFrame()
pl_full = pd.DataFrame()
coaches_full = pd.DataFrame()
games_full = pd.DataFrame() # даты игр

for season in range(2010,2011,1):
    file_path = './input/nhl-data/' + str(season) + 'FullDataset.pkl'
    print(file_path)
    with open(file_path, 'rb') as f:
        game_data = pickle.load(f)
    
    df, pl, st_pl, st_gl, coaches, games = pickle_data(game_data)
    print('full db update: ', season)
    
    df_full = pd.concat([df_full, df])
    st_pl_full = pd.concat([st_pl_full, st_pl])
    st_gl_full = pd.concat([st_gl_full, st_gl])
    pl_full = pd.concat([pl_full, pl])
    coaches_full = pd.concat([coaches_full, coaches])
    games_full = pd.concat([games_full, games])

In [ ]:
df_full.to_csv('df_old.csv', index=False)
st_pl_full.to_csv('players_stats_old.csv', index=False)
st_gl_full.to_csv('goalies_stats_old.csv', index=False)
pl_full.to_csv('players_old.csv', index=False)
coaches.to_csv('coaches_old.csv', index=False)